In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as ss
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

####
def delete_outliers (df, column_name): # Функция удаления выбросов из датафрейма
    '''
    Parameters
    ----------
    df: source DataFrame from which outliers are to be removed
    column_name: name of the column to be checked for outliers
    
    Return: Dataframe without outliers in column_name
    '''
    q1=df[column_name].quantile(0.25)
    q3=df[column_name].quantile(0.75)
    d=1.5*(q3-q1)
    lower_bound=q1-d
    upper_bound=q3+d
    print (f"Найдено и удалено {df[(df[column_name]<lower_bound)|(df[column_name]>upper_bound)].shape[0]} выброс(а/ов) в столбце {column_name}")
    return df[(df[column_name]>=lower_bound)&(df[column_name]<=upper_bound)] # включая концы

def cmp_interval(m, a, b, text): # Функция для сравнения среднего значения m с интервалом [а, b]
    if (m<a) | (m>b):
        text+=" не"
    output_str="Среднее значение " + text + " попадает в доверительный интервал"
    return output_str

def shapiro_result(a, x, text):
    sr=ss.shapiro(x)
    p=sr.pvalue
    f=f"Результат теста с использованием критерия Шапиро-Уилка {text}: statistic={sr.statistic:.3f}, p-value={p:.5f}\n"
    if p<a:
        f+="p-value<alpha -> Распределение не является нормальным\n"
    else:
        f+="p-value>=alpha -> Распределение является нормальным\n"
    return f

def anderson_result(a, x, text):
    a=a*100
    ar=ss.anderson(x)
    s=ar.statistic
    cv=ar.critical_values
    sl=ar.significance_level
    if (a in sl):
        ccv=cv[sl.tolist().index(a)]
        f=f"Результат теста с использованием критерия Андерсона-Дарлинга {text}: statistic={s:.3f}, critical_values={cv}, significance level={sl}\n"
        if (s>ccv):
            f+="Значение статистики превышает критическое значение -> Распределение не является нормальным\n"
        else:
            f+="Значение статистики не превышает критическое значение -> Распределение является нормальным\n"
        return f
    else:
        return f"Не удалось найти критическое значение для уровня значимости {a}%"

def ks_result(a, x, text):
    loc, scale = ss.norm.fit(x) # Определяем параметры распределения
    norm_dist = ss.norm(loc=loc, scale=scale) # Создаем нормальное распределение с данными параметрами
    kr=ss.kstest(x, norm_dist.cdf)
    p=kr.pvalue
    f=f"Результат теста Колмогорова-Смирнова {text}: statistic={kr.statistic:.3f}, p-value={p:.5f}, statistic location={kr.statistic_location:.3f}, statistic sign={kr.statistic_sign:.3f}\n"
    if p<a:
        f+="p-value<alpha -> Распределение не является нормальным\n"
    else:
        f+="p-value>=alpha -> Распределение является нормальным\n"
    return f

def mannwhitneyu_result(a, x, y, text):
    mr=ss.mannwhitneyu(x, y)
    p=mr.pvalue
    f=f"Результат теста с использованием критерия Манна-Уитни {text}: statistic={mr.statistic:.3f}, p-value={p:.5f}\n"
    if p<a:
        f+="p-value<alpha -> Формы распределения не совпадают\n"
    else:
        f+="p-value>=alpha -> Формы распределения совпадают\n"
    return f


def alt_method(x, text): # Дополнительные проверки на нормальность распределения
    mean=x.mean()
    median=x.median()
    d=mean-median
    skew=ss.skew(x)
    kurtosis=ss.kurtosis(x)
    if d!=0:
        cmp="отличается от медианного"
    else:
        cmp_str="равно медианному"
    return f"Среднее значение {text} ({mean:.2f}) {cmp} ({median:.2f}), скошенность: {skew:.2f}, эксцесс:{kurtosis:0.2f}"
####


#dataset_link="Bike Sharing Original Data.csv" # для чтения с диска
dataset_link=r"https://raw.githubusercontent.com/YBI-Foundation/Dataset/refs/heads/main/Bike%20Sharing%20Original%20Data.csv" # для чтения напрямую с репозитория
df=pd.read_csv(dataset_link, sep=",")
print (f"\nИсходный датафрейм:\n{df}\n")

# Проверка на пропуски
n_miss=df.isna().sum().sum()
print (f"Найдено {n_miss} пропусков")
if n_miss>0: # Удаление пропусков
    df.dropna(inplace=True)
    print("Пропуски были удалены")

df2=df.copy() # Копия датафрейма без пропусков для сравнения двух групп
df=df[df['Year']==2012] # Ограничим выборку 2012 годом

# Удаление выбросов из необходимых столбцов
df=delete_outliers(df, 'Wind Speed')
df=delete_outliers(df, 'Temperature')
df=delete_outliers(df, 'Humidity')
df=delete_outliers(df, 'Rider')

print (f"\nПодготовленный датафрейм:\n{df}")

confidence=0.95 # Уровень доверия
print (f"\nУровень доверия: {confidence:0.2f}")
alpha=np.round(1-confidence, 2)
print (f"Уровень значимости (вероятность ошибки первого рода): {alpha:0.2f}")

# Вычисление средних значений, дисперсий и кол-ва элементов
s_Wind_Speed=df['Wind Speed']
s_Rider=df['Rider']

mean_Wind_Speed=s_Wind_Speed.mean()
mean_Rider=s_Rider.mean()
print (f"Среднее значение скорости ветра: {mean_Wind_Speed:0.2f}")
print (f"Среднее значение райдера: {mean_Rider:0.2f}")

var_Wind_Speed=s_Wind_Speed.var()
var_Rider=s_Rider.var()

n_Wind_Speed=s_Wind_Speed.count()
n_Rider=s_Rider.count()

# Определение доверительных интервалов для Wind Speed и Rider
ci_Wind_Speed=ss.norm.interval (confidence= confidence, loc=mean_Wind_Speed, scale=np.sqrt(var_Wind_Speed/n_Wind_Speed))
ci_Rider=ss.norm.interval (confidence= confidence, loc=mean_Rider, scale=np.sqrt(var_Rider/n_Rider))
print (f"{confidence*100}% доверительный интервал для средней скорости ветра: [{ci_Wind_Speed[0]:0.2f} , {ci_Wind_Speed[1]:0.2f}]")
print (f"{confidence*100}% доверительный интервал для среднего райдера: [{ci_Rider[0]:0.2f} , {ci_Rider[1]:0.2f}]")

# Выводы
print (cmp_interval(mean_Wind_Speed, ci_Wind_Speed[0], ci_Wind_Speed[1], "скорости ветра"))
print (cmp_interval(mean_Rider, ci_Rider[0], ci_Rider[1], "райдера"))    
print()

# Проверка на нормальность распределения формальными тестами
print (shapiro_result(alpha, s_Wind_Speed, "для скорости ветра"))
print (shapiro_result(alpha, s_Rider, "для райдера"))
print (anderson_result(alpha, s_Wind_Speed, "для скорости ветра"))
print (anderson_result(alpha, s_Rider, "для райдера"))
print (ks_result(alpha, s_Wind_Speed, "для скорости ветра"))
print (ks_result(alpha, s_Rider, "для райдера"))

# Проверка на нормальность распределения графическим способом
print ("\nГрафики и гистограммы")
fig, ax=plt.subplots(1, 2, figsize=(15, 4))
fig.suptitle('QQ-plot для скорости ветра и райдера')
ss.probplot(s_Wind_Speed, dist='norm', plot=ax[0])
ss.probplot(s_Rider, dist='norm', plot=ax[1])
ax[0].set_title('Скорость ветра')
ax[0].set_xlabel('Теоретические квантили')
ax[0].set_ylabel('Упорядоченные значения')
ax[1].set_title('Райдер')
ax[1].set_xlabel('Теоретические квантили')
ax[1].set_ylabel('Упорядоченные значения')
plt.show()

fig, ax=plt.subplots(ncols=2, figsize=(15, 4))
fig.suptitle('Гистограммы для скорости ветра и райдера')
sns.histplot(data=s_Wind_Speed, bins=25, kde=True, ax=ax[0])
sns.histplot(data=s_Rider, bins=25, kde=True, ax=ax[1])
ax[0].set_xlabel('Скорость ветра')
ax[0].set_ylabel('Количество')
ax[1].set_xlabel('Райдер')
ax[1].set_ylabel('Количество')
plt.show()

print ("Наблюдаем положительную ассиметрию у распределения скорости ветра и отрицательную у райдера")
print (alt_method(s_Wind_Speed, "скорости ветра"))
print (alt_method(s_Rider, "райдера"))

# Группа 2: Скорость ветра в 2011 году
print ("\nВ качестве второй группы наблюдений возьмем данные о скорости ветра за 2011 год и удалим выбросы")
df2=df2[df2['Year']==2011] 
df2=delete_outliers(df2, 'Wind Speed')
s_Wind_Speed2=df2['Wind Speed']

# Проверка на нормальность распределения второй группы формальными тестами
print("\nФормальные тесты для второй группы:")
print (shapiro_result(alpha, s_Wind_Speed2, "для скорости ветра (2011 год)"))
print (anderson_result(alpha, s_Wind_Speed2, "для скорости ветра (2011 год)"))
print (ks_result(alpha, s_Wind_Speed2, "для скорости ветра (2011 год)"))

#Тест Манна-Уитни
print ("Итак, согласно результатам 3 (из 3) формальных тестов распределение в первой группе не является нормальным,\nво второй группе 2 теста подтверждают нормальное распределение и 1 опровергает")
print (mannwhitneyu_result(alpha, s_Wind_Speed, s_Wind_Speed2, "для скорости ветра в 2012 и 2011 годах"))

#Гистограммы для скорости ветра в 2012 и 2011 годах
fig, ax=plt.subplots(ncols=2, figsize=(15, 4))
fig.suptitle('Гистограммы для скорости ветра в 2012 и 2011 годах')
sns.histplot(data=s_Wind_Speed, bins=25, kde=True, ax=ax[0])
sns.histplot(data=s_Wind_Speed2, bins=25, kde=True, ax=ax[1])
ax[0].set_xlabel('Скорость ветра (2012 год)')
ax[0].set_ylabel('Количество')
ax[1].set_xlabel('Скорость ветра (2011 год)')
ax[1].set_ylabel('Количество')
plt.show()

print ("Оценим взаимосвязь между скоростью ветра и райдером для первой выборки")
plt.figure(figsize=(15, 4))
plt.scatter(s_Wind_Speed, s_Rider)
plt.title("Скорость ветра / Райдер")
plt.xlabel("Скорость ветра")
plt.ylabel("Райдер")
plt.show()
print (f"Корреляция между скоростью ветра и райдером: {s_Wind_Speed.corr(s_Rider):.2f}")
print ("На основании графика и вычисленной корреляции заключаем, что взаимосвязь между скоростью ветра и райдером слабая")

sns.lmplot(x='Wind Speed', y='Rider', data=df)
plt.title("Оценка линейной регрессии")
plt.xlabel("Скорость ветра")
plt.ylabel("Райдер")
plt.show()

x=sm.add_constant(s_Wind_Speed)
model=sm.OLS(s_Rider, x) # Создание модели линейной регрессии с использованием метода наименьших квадратов
print (f"Результаты оценки линейной регрессии:\n{model.fit().summary()}")
print ("\nИз результатов считываем детерминированную часть уравнения: Rider = 6583.9412 - 73.6114 * Wind_Speed")
print ("Таким образом, так как коэффициент перед Wind_Speed строго отрицателен, можем прогнозировать некоторое уменьшение райдера при росте скорости ветра.")
print ("В то же время стоит помнить, что найденная взаимосвязь слабая, а райдер зависит и от других параметров. В частности, в предыдущем домашнем задании была отмечена зависимость от температуры и сезонности")